In [1]:
import molsysmt as msm
import pyunitwizard as puw

In [2]:
import openmm as mm
from openmm import unit
from openmm import app
from tqdm import tqdm
import numpy as np

In [3]:
dialanine = msm.build.build_peptide('AceAlaNme')

In [4]:
msm.structure.center(dialanine, in_place=True)

In [5]:
dialanine = msm.build.solvate(dialanine, box_shape='cubic',
                              clearance='14.0 angstroms')

In [6]:
msm.info(dialanine)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_peptides,n_structures
molsysmt.MolSys,2050,679,677,2,677,2,676,1,1


In [7]:
topology = msm.convert(dialanine, to_form='openmm.topology')

In [8]:
positions = msm.get(dialanine, coordinates=True)
positions = puw.convert(positions[0], to_form='openmm.unit')

In [9]:
forcefield = app.ForceField("amber14-all.xml", "amber14/tip3p.xml")

In [10]:
system = forcefield.createSystem(topology, nonbondedMethod=app.PME, nonbondedCutoff=1.2*unit.nanometers, constraints=app.HBonds)

In [11]:
#system.addForce(app.MonteCarloBarostat(1*unit.atmosphere, 400*kelvin, 25))

In [12]:
temperature = 400*unit.kelvin
friction = 1.0/unit.picosecond
timestep = 2.0*unit.femtoseconds
integrator = mm.LangevinIntegrator(temperature, friction, timestep)

In [13]:
platform = mm.Platform.getPlatformByName('CUDA')

In [14]:
simulation = app.Simulation(topology, system, integrator, platform)
simulation.context.setPositions(positions)

In [15]:
simulation.minimizeEnergy()

In [16]:
simulation.context.setVelocitiesToTemperature(temperature)

In [17]:
md_time = 500*unit.nanoseconds
md_steps = int(round(md_time/timestep))
saving_time = 10*unit.picoseconds
saving_steps = int(round(saving_time/timestep))

In [18]:
report_interval = int(round(1.0*unit.nanoseconds/timestep))
tqdm_reporter = msm.thirds.openmm.reporters.TQDMReporter(report_interval, md_steps)

In [19]:
report_interval = saving_steps
h5msm_reporter = msm.thirds.openmm.reporters.H5MSMReporter('traj.h5msm', report_interval, md_steps)

In [20]:
simulation.reporters+=[tqdm_reporter, h5msm_reporter]

In [21]:
simulation.step(md_steps)

  0%|          | 0/250000000 [00:00<?, ?it/s]


Potential energy: -24025.93 kJ/mol ± 464.30 kJ/mol
Temperature: 400.44 K ± 8.62 K

Execution time: 0 days, 12 hours, 13 minutes, and 37.03 seconds (981.438 ns/day).



In [22]:
h5msm_reporter.close()